# Notebook to classify mutants output by the variant calling pipeline for any scenario where the sample_id is of the form GSM_BARCODE

In [ ]:
import numpy as np
import os
import re
import pandas as pd
import glob
import matplotlib.pyplot as plt
from collections import Counter
import pickle

In [ ]:
md = pd.read_pickle("./Metadata/parkinsons_metadata.pkl")
md.head()

In [ ]:
donor_dict = dict(zip(md.GSM_id, md.donor_id)) # Links GSM to a useful unique donor identifier from the metadata
gsm_dict = dict(zip(md.SRR_id, md.GSM_id)) # Links SRR to GSM

Merges all the depth files into a single Dataframe

In [ ]:
depth_files = glob.glob(f"./Data/variant/*/*depth.csv") # List of all donor files from all GSMs
depth_frames = (pd.read_csv(f) for f in depth_files)
depth_all  = pd.concat(depth_frames, ignore_index=False, sort = False)
print('Done!')
depth_all.shape

In [ ]:
depth_all.head()

The below splits barcode from GSM and then maps it to donor_id. If the mapping is not 1-1 then an additional map must be made to keep this correct

In [ ]:
base_dict = dict(zip(depth_all.sample_id,depth_all.bases_passed))

depth_all['GSM'] = depth_all.sample_id.str.split('_',n = 1, expand = True)[0]
depth_all['barcode'] = depth_all.sample_id.str.split('_',n = 1, expand = True)[1]

depth_all['donor_id'] = depth_all['GSM'].map(donor_dict)

depth_all.to_pickle(f'./Data/depthAll.pkl')

Set a threshold on the bases passed based off the plot generated below - good quality cells should have a roughly log-normal distribution on the number of bases passed so try to cut off any which skew this distribution. Anything with fewer than 100 bases passing should be treated cautiously

In [ ]:
bin_arr = [0]
bin_arr.extend(np.logspace(np.log10(1),np.log10(100000), 50))
plt.hist((depth_all['bases_passed']),bins=bin_arr)
plt.xscale('log');

In [ ]:
threshold = 300
passed_samples = depth_all[depth_all['bases_passed']>threshold]

# This is the file needed to re-filter the expression matrix
np.savetxt('mitochondrialFilteredBarcodes.txt', passed_samples.drop_duplicates['sample_id'], fmt="%s")

variant_files = []
coverage_files = []
variant_files.extend(list('./Data/variant/' + passed_samples['GSM'] + '/' +passed_samples['sample_id'] + '_variants.csv'))
coverage_files.extend(list('./Data/variant/'  + passed_samples['GSM'] + '/' + passed_samples['sample_id'] + '_coverage.pkl'))

Merges all the coverage files passing quality control into a single frame

In [ ]:
coverage=pd.DataFrame(index=list(range(0,16569))) # Edit this to genome length of species

# The below creates a new column for every coverage file by mapping the index to the coverage of that cell
for f in coverage_files:
    huh = pd.read_pickle(f)
    coverage[f.split('/')[-1][:-13]] = coverage.index.map(huh)
    
# Creates a multi-index column for easy reference to donor and barcode
cell_ids = pd.DataFrame(coverage.columns).T
coverage.columns = pd.MultiIndex.from_frame(pd.DataFrame(cell_ids.iloc[0].str.split('_',n=0, expand=True)).rename(columns={0:'donor_id',1:'sample_id'}))

coverage=coverage.sort_values(by=['donor_id','sample_id'],  axis=1)

coverage.to_pickle('./Data/coverage.pkl')

In [ ]:
# Set this to whatever depth threshold you used for variant calling (default 200)
coverage_lim=200

# Creates a dictionary mapping every position to the number of cells 
# from a donor which covered that position at sufficient depth for variant calling
cell_base={}
for GSM in md.GSM_id.unique().astype(str):
    
    split=coverage.xs(GSM, level='donor_id', axis=1).fillna(0) > coverage_lim
    
    cell_base.update({GSM+'_'+str(k) : v for k, v in dict(split.sum(axis=1)).items()})

In [ ]:
variant_frames = (pd.read_csv(f) for f in variant_files)
d_donor_all  = pd.concat(variant_frames, ignore_index=False, sort = False)
print('Done!')
d_donor_all.shape

In [ ]:
# This splits the GSM from the barcode and then maps it to the donor id
# Only works for 1-1 mapping of donor->GSM (is fine for multiple GSMs per donor)

d_donor_all['GSM'] = d_donor_all["sample_id"].str.split('_',n = 1, expand = True)[0]
d_donor_all['donor_id'] = d_donor_all['GSM'].map(donor_dict)

In [ ]:
# Maps sample id to how many bases passed the filter

base_dict = dict(zip(depth_all.sample_id,depth_all.bases_passed))
d_donor_all['bases_passed'] = d_donor_all.sample_id.map(base_dict)

In [ ]:
d_donor_all['MUT'] = d_donor_all['REF']+d_donor_all['POS'].apply(str)+d_donor_all['ALT']
d_donor_all['donor_mut'] = d_donor_all['GSM'].astype(str)+d_donor_all['MUT']
d_donor_all['donor_POS'] =  d_donor_all['GSM'].astype(str)+'_'+d_donor_all['POS'].astype(str)
d_donor_all['cells_possible'] = d_donor_all.donor_POS.map(cell_base)

The following splits up mutations from each donor and labels the average heteroplasmy they appear at (given that they do appear), the standard deviation of heteroplasmy and the proportion of cells from a donor they appear in

In [ ]:
mutant_stats = pd.DataFrame()

mutant_group = d_donor_all[['donor_id','donor_mut','HF','MUT','sample_id','cells_possible']].groupby(['donor_id','MUT','donor_mut'])
mutant_stats['average'] = mutant_group['HF','donor_mut'].mean()['HF']
mutant_stats['standardD'] = mutant_group['HF','donor_mut'].std()['HF']
mutant_stats['mutant_sequenced_prop'] = (mutant_group['sample_id'].count()/mutant_group['cells_possible'].mean())

mean_het = dict(zip(mutant_stats.index.get_level_values(level='donor_mut'),mutant_stats.average))
cell_prop = dict(zip(mutant_stats.index.get_level_values(level='donor_mut'),mutant_stats.mutant_sequenced_prop))
std_dict = dict(zip(mutant_stats.index.get_level_values(level='donor_mut'),mutant_stats.standardD))

d_donor_all['mutant_sequenced_prop']=d_donor_all['donor_mut'].map(cell_prop)
d_donor_all['mean_HF']=d_donor_all['donor_mut'].map(mean_het)
d_donor_all['mutant_stds']=d_donor_all['donor_mut'].map(std_dict)

In [ ]:
# Creates a dictionary of GSM to number of cells
donor_cell_dict={}
for GSM in d_donor_all.GSM.unique():
    donor_cell_dict[GSM] = len(passed_samples[passed_samples.GSM == GSM])

In [ ]:
d_donor_all['base_cell_prop'] = d_donor_all.cells_possible/d_donor_all.GSM.map(donor_cell_dict) # Proportion of all donor cells the base is covered by
d_donor_all['mutant_donor_prop'] = d_donor_all.base_cell_prop*d_donor_all.mutant_sequenced_prop # Proportion of all donor cells mutant is found in

This identifies any mutations which are not haplotype or cryptic, and appear in all donors from the experiment

In [ ]:
# Dictionary of all mutations which are not haplotype (mean_HF>0.95) by donor
donor_no_haps={}
for donor in d_donor_all.donor_id.unique():
    donor_no_haps[donor] = mutant_stats.loc[donor][mutant_stats.loc[donor]['average']<0.95]

# Creates a list of all mutations found in each donor and then counts how many times they appear in the list
mutations=[]
for donor in d_donor_all.donor_id.unique():
    mutantions.extend(donor_no_haps[donor].index.get_level_values(level='MUT'))
uni = Counter(mutations)

# Creates list of all mutations which appear in more than three donors
common_mut=[]
for key in uni: 
    if uni[key] >3:
        common_mut.append(key)

Creates column ['Mutant_type'] to identify every mutation

In [ ]:
# Find all the mutations common across all donors
common = d_donor_all[d_donor_all['MUT'].isin(common_mut)]
common['type'] = 'Common mutation'
common_dict=dict(zip(common.donor_mut,common.type))

# Find all the cryptic mutations
singletons = d_donor_all.drop_duplicates(subset = ['donor_mut'], keep = False)
singletons['single']='Cryptic'
single_dict = dict(zip(singletons.donor_mut,singletons.single))

# Find Haplotype Mutations
others = d_donor_all[((~d_donor_all['donor_mut'].isin(singletons['donor_mut']))&(~d_donor_all['donor_mut'].isin(list(common_dict.keys()))))]
fully_sequenced = others[((others.mutant_sequenced_prop> 0.95))]
fully_sequenced['full'] = 'Full Sequenced'
fully_sequenced_dict = dict(zip(fully_sequenced.donor_mut,fully_sequenced.full))

# Find Inherited Mutations
partial_sequenced = others[~others['donor_mut'].isin(fully_sequenced['donor_mut'])]
partial_sequenced['hap'] = 'Part Sequenced'
part_dict = dict(zip(partial_sequenced.donor_mut,partial_sequenced.hap))

# Label all the mutation types in the full frame
mut_type_dict = {**common_dict , **single_dict, **fully_sequenced_dict, **part_dict}
d_donor_all['mutant_type'] = d_donor_all['donor_mut'].map(mut_type_dict)

d_donor_all = d_donor_all.drop(columns=['donor_mut','donor_POS']).reset_index(drop=True)

The following is only applicable to human data - it queries every mutation against a database which can predict how pathological it is going it be

In [ ]:
scores = pickle.load(open('mutPred_probV1.pkl', 'rb'))
classes = pickle.load(open('mutPred_predV1.pkl', 'rb'))
aachange = pickle.load(open('aachangeV1.pkl', 'rb'))
haplomap = pickle.load(open('haplomapV1.pkl', 'rb'))

In [ ]:
d_donor_all['MutPred_Probability'] = d_donor_all['MUT'].map(scores)
d_donor_all['MutPred_Prediction'] = d_donor_all['MUT'].map(classes)
d_donor_all['AAchange'] = d_donor_all['MUT'].map(aachange)
d_donor_all['haplotypes'] = d_donor_all['MUT'].map(haplomap)
d_donor_all = d_donor_all.drop(columns=['MUT']).reset_index(drop=True)

This labels protein coding mutations depending on the effect they have

In [ ]:
for i, aa in enumerate(d_donor_all['AAchange']):
    if pd.isna(d_donor_all.loc[i]['AAchange']):
        continue
    elif d_donor_all.loc[i]['AAchange'][0] == '.':
        continue
    elif d_donor_all.loc[i]['AAchange'][0] == d_donor_all.loc[i]['AAchange'][-1]:
        d_donor_all.loc[i,'MutPred_Probability'] = 0
        d_donor_all.loc[i,'MutPred_Prediction'] = 'Synonymous'
    elif ((d_donor_all.loc[i]['AAchange'][0] == 'X') or (d_donor_all.loc[i]['AAchange'][-1] == 'X')):
        d_donor_all.loc[i,'MutPred_Prediction'] = 'Non-Synonymous Stop Substitution'

In [ ]:
d_donor_all.to_pickle('./Data/variants.pkl')